## Adaptation de namedGame à Jupyter

Vous pouvez utiliser cette feuille pour visualiser un match.

Pour charger vos joueurs, il faut changer leurs noms dans 'classNames' (ligne 13)

In [1]:
''' Sorry no comments :).
'''
import Puissance_4
import importlib
import time
from io import StringIO
import sys
from IPython.display import display, HTML, clear_output

# Uncomment for saving the successive SVG to PNG
# from cairosvg import svg2png

# Define here the players to load
classNames = ['randomPlayer', 'randomPlayer']

b = Puissance_4.Board()

players = []
player1class = importlib.import_module(classNames[0])
player1 = player1class.myPlayer()
player1.newGame(Puissance_4.Board._RED)
players.append(player1)

player2class = importlib.import_module(classNames[1])
player2 = player2class.myPlayer()
player2.newGame(Puissance_4.Board._YELLOW)
players.append(player2)

totalTime = [0,0] # total real time for each player
nextplayer = 0
nextplayercolor = Puissance_4.Board._RED
nbmoves = 1

outputs = ["",""]
sysstdout= sys.stdout
stringio = StringIO()
wrongmovefrom = 0

print("The game is setup with " + player1.getPlayerName() + " playing RED against " + player2.getPlayerName() + " playing YELLOW")
print("Engines loaded. Ready to go.")


The game is setup with Random Player playing RED against Random Player playing YELLOW
Engines loaded. Ready to go.


In [2]:
coup = 1
while not b.is_game_over():
    print("Referee Board")
    clear_output(wait=True)
    #display(HTML(b.svg()))
    display(HTML(b.svg2()))
    #svg2png(bytestring=b.svg(),write_to='output-'+str(coup).zfill(4)+'.png')
    legals = b.legal_moves() # legal moves are given as internal (flat) coordinates, not A1, A2, ...

    print("Legal Moves: ", [b.coord_to_name(m) for m in legals]) # I have to use this wrapper if I want to print them
    nbmoves += 1
    otherplayer = (nextplayer + 1) % 2
    othercolor = Puissance_4.Board.flip(nextplayercolor)
    
    currentTime = time.time()
    sys.stdout = stringio
    move = players[nextplayer].getPlayerMove() # The move must be given by "A1", ... "J8" string coordinates (not as an internal move)
    sys.stdout = sysstdout
    playeroutput = stringio.getvalue()
    stringio.truncate(0)
    stringio.seek(0)
    print(("[Player "+str(nextplayer) + "] ").join(playeroutput.splitlines(True)))
    outputs[nextplayer] += playeroutput
    totalTime[nextplayer] += time.time() - currentTime
    print("Player ", nextplayercolor, players[nextplayer].getPlayerName(), "plays: " + str(move)) #changed 

    if not Puissance_4.Board.name_to_coord(move) in legals:
        print(otherplayer, nextplayer, nextplayercolor)
        print("Problem: illegal move")
        wrongmovefrom = nextplayercolor
        break
    b.push(Puissance_4.Board.name_to_coord(move)) # Here I have to internally flatten the move to be able to check it.
    players[otherplayer].playOpponentMove(move)
 
    nextplayer = otherplayer
    nextplayercolor = othercolor
    time.sleep(1)
    coup += 1

print("Referee Board:")
clear_output(wait=True)
display(HTML(b.svg()))
print("The game is over")
b.board_to_text(b._board)
result = b.result()
print("Time:", totalTime)
print("Winner: ", end="")
if wrongmovefrom > 0:
    if wrongmovefrom == b._YELLOW:
        print("RED")
    elif wrongmovefrom == b._RED:
        print("YELLOW")
    else:
        print("ERROR")
elif result == "0-1":
    print("YELLOW")
elif result == "1-0":
    print("RED")
else:
    print("DEUCE")

# To convert png files to animated gif    
# convert -delay 10 -loop 0 *.png -background white -alpha remove -alpha off output.gif



The game is over
Time: [0.0019979476928710938, 0.0]
Winner: RED
